顺序块

In [18]:
import torch
from torch import nn
from torch import functional as F

In [19]:
class MySequential(nn.Module):
    def __init__(self, *args):
        super().__init__()
        for idx, module in enumerate(args):
            # Module类自带一个成员变量_modules,类型是OrderedDict
            self._modules[str(idx)] = module
            
    def forward(self, X):
        for block in self._modules.values():    # 字典的value就是那些module
            X = block(X)
        return X
            

In [20]:
net = MySequential(nn.Linear(20, 256), nn.ReLU(), nn.Linear(256, 10))
X = torch.rand(2, 20)
net(X)

tensor([[-0.0121, -0.0479, -0.1390, -0.0256, -0.1550,  0.2412, -0.0868,  0.2041,
          0.0157,  0.0513],
        [-0.0029, -0.1223, -0.1374, -0.1804, -0.2017,  0.1549, -0.1933,  0.2600,
         -0.1078,  0.1702]], grad_fn=<AddmmBackward0>)

5.2参数管理

In [21]:
import torch
from torch import nn

In [22]:
class share_MLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.linear = nn.Linear(4, 4)
        self.net = nn.Sequential(
                self.linear,
                nn.ReLU(),
                self.linear)
    def forward(self, x):
        return self.net(x)
    

In [23]:
input_data = torch.rand(4, 4)
target_data = torch.rand(4, 4)
mlp = share_MLP()
output_data = mlp.forward(input_data)

In [24]:
loss = nn.MSELoss()
lr = 0.1
optim = torch.optim.SGD(mlp.parameters(), lr = lr)

In [25]:
num_epochs = 5

In [26]:
def train(net, loss, num_epochs, optim): 
    for epoch in range(num_epochs):
        
        l = loss(output_data, target_data)
        optim.zero_grad()
        l.backward()
        print('weight of layer1', net.net[0].weight.data)
        print('weight of layer2', net.net[2].weight.data)
        print("grad of layer1's weight", net.net[0].weight.grad)
        print("grad of layer2's weight", net.net[2].weight.grad)
        optim.step()

In [27]:
train(mlp, loss, num_epochs, optim)

weight of layer1 tensor([[-0.4021, -0.1951, -0.2522, -0.4914],
        [-0.1438,  0.2135, -0.4457,  0.4051],
        [-0.2398,  0.2357, -0.4527, -0.1559],
        [-0.2104, -0.1944, -0.3462, -0.0373]])
weight of layer2 tensor([[-0.4021, -0.1951, -0.2522, -0.4914],
        [-0.1438,  0.2135, -0.4457,  0.4051],
        [-0.2398,  0.2357, -0.4527, -0.1559],
        [-0.2104, -0.1944, -0.3462, -0.0373]])
grad of layer1's weight tensor([[ 0.0000,  0.0000,  0.0000, -0.0070],
        [ 0.0000,  0.0000,  0.0000, -0.0073],
        [ 0.0000,  0.0000,  0.0000, -0.0090],
        [ 0.0009,  0.0099,  0.0126,  0.0106]])
grad of layer2's weight tensor([[ 0.0000,  0.0000,  0.0000, -0.0070],
        [ 0.0000,  0.0000,  0.0000, -0.0073],
        [ 0.0000,  0.0000,  0.0000, -0.0090],
        [ 0.0009,  0.0099,  0.0126,  0.0106]])


RuntimeError: Trying to backward through the graph a second time (or directly access saved tensors after they have already been freed). Saved intermediate values of the graph are freed when you call .backward() or autograd.grad(). Specify retain_graph=True if you need to backward through the graph a second time or if you need to access saved tensors after calling backward.